In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

from tqdm import tqdm

In [44]:
people = requests.get("https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%BC%D0%B5%D0%BD%D0%B5%D0%B2,_%D0%9B%D0%B5%D0%B2_%D0%91%D0%BE%D1%80%D0%B8%D1%81%D0%BE%D0%B2%D0%B8%D1%87")
soup = BeautifulSoup(people.content, 'html.parser')

In [67]:
def get_hrefs():
    people_page = requests.get("https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D0%B5%D0%BB%D0%B8_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B3%D0%BE_%D0%B3%D0%BE%D1%81%D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2%D0%B0#%D0%A0%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B5_%D1%86%D0%B0%D1%80%D0%B8")
    soup = BeautifulSoup(people_page.content, 'html.parser')
    pattern = re.compile("\(.*\d+.*\)")
    filtered_links = list(map(lambda x: x.find('a'), filter(lambda x: pattern.findall(x.text) != [], soup.find_all('li'))))
    d = re.compile("\d+")
    filtered_hrefs = list(map(lambda x: x['href'], filter(lambda x: d.findall(x.text) == [], filtered_links)))
    return list(filter(lambda x: "wiki" in x, filtered_hrefs))

def get_list_sentences(text):
    return list(filter(lambda x: x != '',
        re.sub(r'\(.*\)', "", text.replace("\xa0", "").replace("\n", ""))
            .split(".")))

def enrich_result(result_list, sentences):
    if sentences is not None:
        if len(sentences) >= 3:
            sentences = sentences[:3]
            result_list[0].append(''.join(map(str, sentences)))
            result_list[5].append(''.join(map(str, sentences[::-1])))
            result_list[4].append(''.join(map(str, sentences[:2])))
            result_list[3].append(sentences[2])
    else:
        print("miss")

def crawl_by_links(hrefs):
    final_result = {0: [], 5: [], 4: [], 3: []}
    for href in tqdm(hrefs):
        try:
            final_href = f"https://ru.wikipedia.org{href}"
            people = requests.get(final_href)
            soup = BeautifulSoup(people.content, 'html.parser')
            text = ''.join(map(str, list(map(lambda x: x.text, soup.find_all("p")[:2]))))
            sentences = get_list_sentences(text)
            enrich_result(final_result, sentences)
        except:
            print(final_href)
            print(text)
            print("============================")
    return pd.DataFrame.from_dict(final_result)


In [28]:
filtered_hrefs = get_hrefs()

In [68]:
final_pd_result = crawl_by_links(filtered_hrefs)


100%|██████████| 132/132 [01:23<00:00,  1.58it/s]


In [69]:
final_pd_result.to_csv("./data/test.csv", index=False)

pd.read_csv("./data/test.csv")



,0,5,4,3
0,"Вели́кое кня́жество Влади́мирское, до 1157года...",С середины XIII века сюзеренитет великих княз...,"Вели́кое кня́жество Влади́мирское, до 1157года...",С середины XIII века сюзеренитет великих княз...
1,Новгоро́дская респу́блика — севернорусское сре...,Находясь с 1245 года под сюзеренитетом велики...,Новгоро́дская респу́блика — севернорусское сре...,Находясь с 1245 года под сюзеренитетом велики...
2,Вели́кое кня́жество Лито́вское— восточноевропе...,С 1385 года находилось в личной унии с Королев...,Вели́кое кня́жество Лито́вское— восточноевропе...,С 1385 года находилось в личной унии с Королев...
3,Вели́кое кня́жество Моско́вское— средневековое...,Возвышению Москвы и укреплению её авторитета ...,Вели́кое кня́жество Моско́вское— средневековое...,Возвышению Москвы и укреплению её авторитета ...
4,Оле́г — князь Новгородский с 879 года и велики...,Получив власть над новгородскими землями посл...,Оле́г — князь Новгородский с 879 года и велики...,
5,"Святосла́в Яросла́вич, в крещении Николай[1] ...","Второй раз в 1054 году, когда Ярослав поделил...","Святосла́в Яросла́вич, в крещении Николай[1] ...","Второй раз в 1054 году, когда Ярослав поделил..."
6,Святопо́лк В 1069 году отец посадил Святополка...,"В 1071 году Всеслав Брячиславич, прежний княз...",Святопо́лк В 1069 году отец посадил Святополка...,"В 1071 году Всеслав Брячиславич, прежний княз..."
7,"Влади́мир Все́володович Монома́х , государстве...",Владимир Мономах был сыном князя Всеволода Яро...,"Влади́мир Все́володович Монома́х , государстве...",Владимир Мономах был сыном князя Всеволода Яро...
8,Яропо́лк Влади́мирович — сын Владимира Мономах...,"Ярополк родился, вероятнее всего, в Чернигове,...",Яропо́лк Влади́мирович — сын Владимира Мономах...,"Ярополк родился, вероятнее всего, в Чернигове,..."
9,Все́волод О́льгович Всеволод был старшим сыном...,В 1111 году он вместе с другими князьями прин...,Все́волод О́льгович Всеволод был старшим сыном...,В 1111 году он вместе с другими князьями прин...


In [57]:
h = "https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D0%BB%D0%B8%D0%BA%D0%BE%D0%B5_%D0%BA%D0%BD%D1%8F%D0%B6%D0%B5%D1%81%D1%82%D0%B2%D0%BE_%D0%9B%D0%B8%D1%82%D0%BE%D0%B2%D1%81%D0%BA%D0%BE%D0%B5"

people = requests.get(h)
soup_test = BeautifulSoup(people.content, 'html.parser')

'Михаи́л Все́володович (известный также, как Михаи́л Черни́говский, 1179—1246)\xa0— князь Переяславский (1206), Новгородский (1224, 1229), Черниговский (1223—1246), Галицкий (1235—1239), Великий князь Киевский (1238—1239, 1241—1243). Сын Всеволода Чермного и дочери польского князя Казимира II. Причислен к лику святых на Соборе 1547 года; дни памяти\xa0— 20\xa0сентября (3\xa0октября) и 14\xa0(27)\xa0февраля.\nПосле изгнания отцом из Переяславля сына Всеволода Большое Гнездо Ярослава ненадолго занял переяславский престол.\n'

In [58]:
text_test = ''.join(map(str, list(map(lambda x: x.text, soup_test.find_all("p")[:2]))))

list(filter(lambda x: x != '',
    re.sub(r'\(.*\)', "", text_test.replace("\xa0", "").replace("\n", ""))
        .split(".")))


['Вели́кое кня́жество Лито́вское— восточноевропейское государство, существовавшее с середины XIII века по 1795 год на территории современных Белоруссии ',
 ' Обширные земли Руси, попавшие под власть Великого княжества Литовского и составлявшие львиную долю его территории, именуются Литовской Русью',
 'С 1385 года находилось в личной унии с Королевством Польским, а с 1569 года— в сеймовой Люблинской унии в составе федеративной Речи Посполитой',
 ' В XV—XVI веках Великое княжество Литовское— соперник Великого княжества Московского в борьбе за господство на восточнославянских землях и в целом в Восточной Европе',
 ' Прекратило существование после третьего раздела Речи Посполитой в 1795 году',
 ' К 1815 году вся территория бывшего княжества вошла в состав Российской империи']

In [1]:
# PREPROCESS

In [5]:
!nltk.download('stopwords')

import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy2
from pymystem3 import Mystem
from string import punctuation

stop_words = stopwords.words('russian')
stop_words.remove("не")
stop_words.remove("ни")
mystem = Mystem()
stemmer = SnowballStemmer(language='russian')
morph = pymorphy2.MorphAnalyzer()

/bin/bash: -c: line 0: syntax error near unexpected token `'stopwords''
/bin/bash: -c: line 0: `nltk.download('stopwords')'


In [6]:
text = "Екатери́на I (Ма́рта Самуи́ловна Скавро́нская, в браке Крузе; после принятия православия Екатери́на Алексе́евна Миха́йлова; 5 [15] апреля 1684 — 6 [17] мая 1727, Санкт-Петербург) — российская императрица с 1721 года (как супруга царствующего императора)"

def preprocess(text, is_stem = False, is_morph = True, is_remove_stop_words = True):
    if text is None:
        text = ""
    text = text.lower()
    text = re.sub(r'\[.*?\]|\(.*?\)|\n|,|\.|\xa0|;', "", text).replace(chr(769), "")
    # tokens = word_tokenize(text, language='russian')
    tokens = mystem.lemmatize(text)
    if is_remove_stop_words:
        tokens = [token for token in tokens if token not in stop_words\
            and token != " " and token != '' \
            and token.strip() not in punctuation]

    if is_stem:
        tokens = list(map(lambda x: stemmer.stem(x), tokens))
    if is_morph:
        tokens = list(map(lambda x: morph.parse(x)[0].normal_form, tokens))

    return ' '.join(tokens)

preprocess(text)

'екатерина i   —  российский императрица 1721 год'

In [11]:
import pandas as pd
origin_result_data = pd.read_csv("./data/result.csv")
origin_result_data.head()

,Unnamed: 0,original,scored_text,distance,score
0,0,"Вели́кое кня́жество Влади́мирское, до 1157года...",С середины XIII века сюзеренитет великих княз...,0.923924,5
1,1,Новгоро́дская респу́блика — севернорусское сре...,Находясь с 1245 года под сюзеренитетом велики...,0.908120,5
2,2,Вели́кое кня́жество Лито́вское— восточноевропе...,С 1385 года находилось в личной унии с Королев...,0.861883,5
3,3,Вели́кое кня́жество Моско́вское— средневековое...,Возвышению Москвы и укреплению её авторитета ...,0.901416,5
4,4,Оле́г — князь Новгородский с 879 года и велики...,Получив власть над новгородскими землями посл...,0.916509,5


In [16]:
after_preprocess = origin_result_data["original"].transform(preprocess)

In [17]:
after_preprocess = after_preprocess.dropna()
after_preprocess = after_preprocess.drop_duplicates()

In [18]:
after_preprocess

0     великий княжество владимирский 1157год  —  суз...
1     новгородский республика  —  севернорусский сре...
2     великий княжество литовский —  восточноевропей...
3     великий княжество московский —  средневековый ...
4     олег  —  князь новгородский 879 год великий кн...
5     святослав ярослав крещение николай год : « род...
6     святополк 1069 год отец посадить святополк пол...
7     владимир всеволод моном государственный деятел...
8     ярополк владимир  —  сын владимир моном князь ...
9     всеволод ольгович всеволод старший сын князь о...
10    ростислав мстисла сын мстислав внук владимир м...
11    глеб юрий  —  князь переяславский киевский сын...
12    роман ростисла 1151 год помогать изяслав мстис...
13    рюрик ростиславичпосле смерть отец 1167 год рю...
14    ростислав рюрик происходить время пребывание  ...
15    ярослав рождаться 8 февраль 1191г переяславль ...
16    ростислав мстисла —  занимать смоленский прест...
17    даниил роман ефросинья анныпосле гибель от

In [20]:
with open('/home/droman/Documents/diploma/spacy/data/data_for_labeling_with_preprocess.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(set(after_preprocess.tolist())))


In [21]:
len(set(after_preprocess.tolist()))


54